In [ ]:
%load_ext autoreload
%autoreload 2

# Choose a garment

First, you need to choose a garment to simulate.

Its template and some auxiliary data should be stored in the .pkl file under `DEFAULTS.data_root/aux_data/garment_dicts/`

You can choose from the list of outfits already provided in this folder:

![ccraft_garments](static/ccraft_garments.png)

Or you can import a new garment from an `.obj` file

We also provide `.obj` files for all garments usen in the paper in `DEFAULTS.data_root/aux_data/garment_meshes/` directory.
Note that these `.obj` files only have demonstrational purpose. 
For inference and training we use garment data stored in the .pkl files under `DEFAULTS.data_root/aux_data/garments_dicts/`, not the .obj files

## Add your own garment from an `.obj` file

In this notebook we demonstrate how to start the simulation from arbitrary SMPL(-X) pose using linear blend-skinning (LBS) initialization. If you don't want to use LBS (for example, you are using non-SMPL(-X) body meshes),  plase see `Inference_from_mesh_sequence.ipynb`.

To add a new garment, you should have an `.obj` file with the garment mesh that is **aligned with the canonical body model** (zero-pose and zero-shape) you want to use.

We currently support [SMPL](https://smpl.is.tue.mpg.de/) and [SMPL-X](https://smpl-x.is.tue.mpg.de/) body models.

Note: you will not be able to run LBS-initialized simulation with a body model that is different from the one you specify in this step.

First, create a .pkl file with your garment data using `add_garment_to_garments_dict` function.

It builds a dictionary for the garment that contains:
* `rest_pos`: \[Nx3\], positions of the vertices in canonical pose that are aligned to zero- pose and shape SMPL body.
* `faces`: \[Fx3\], triplets of node indices that constitute each face
* `node_type` \[Nx1\], node type labels (`0` for regular, `3` for "pinned"). By default, all nodes are regular, we show how to add "pinned nodes" later in this notebook
* `lbs` dictionary with shape- and pose- blendshapes and skinning weights for the garment, sampled from SMPL model
* `center` and `coarse_edges` info on long-range (coarse) edges used to build hiererchical graph of the garment.

To be able to start simulation from arbitrary pose, we use linear blend-skinning (LBS) to initialize the garment geometry in the first frame. For each garment node we sample pose- and shape-blendshapes and skinning weights from the closest SMPL(-X) node in canonical pose.

However, for loose garments such approach may result in overly-stretched triangles. Therefore, we use the approach introduced in [\[Santesteban et al. 2021\]](http://mslab.es/projects/SelfSupervisedGarmentCollisions/) and average skinning weights and blendshapes over many randomly sampled SMPL(-X) nodes around the given garment node.

The parameter `n_samples_lbs` controls the number of random samples to use. We recommend setting it to 0 for tight-fitting garments (shirts, pants) and to 1000 for loose ones (skirts, dresses).


In [3]:
import os
from utils.mesh_creation import GarmentCreator, add_pinned_verts
from utils.defaults import DEFAULTS


garment_obj_path = os.path.join(DEFAULTS.aux_data, 'garment_meshes', 'smplx', 'cindy_020_combined.obj')

body_models_root = os.path.join(DEFAULTS.aux_data, 'body_models')

# Model type, either 'smpl' or 'smplx'
model_type = 'smplx'

# gender, either 'male`, `female` or `neutral`
gender = 'female'

garment_dict_path = os.path.join(DEFAULTS.aux_data, 'garments_dicts', model_type, 'cindy_020_combined_test.pkl')


# Use approximate_center=True to create temlate faster. In the original paper code it was False
gc = GarmentCreator(body_models_root, model_type, gender, 
                    n_samples_lbs=0, verbose=True, coarse=True, approximate_center=True)
gc.add_garment(garment_obj_path, garment_dict_path)

# Now garment 'cindy_020_combined_test' is added to the garments_dict.pkl file and can be used in furter steps


Sampling LBS weights...
Done.
Adding coarse edges... (may take a while)
Done.
Garment dict saved to /data/agrigorev/02_Projects/ccraft_data/aux_data/garments_dicts/smplx/cindy_020_combined_test.pkl


### Add pinned vertices

For some gaments, it is necessary to fix positions for a subset of nodes relative to the body. For example, fix the top ring of a skirt or pants to prevent it from falling off the body.

To label a set of garment nodes as "pinned", you need to use `add_pinned_verts` function and provide it with the list of node indices that you want to pin.

One easy way of getting indices for a set of nodes, is by using [Blender](https://www.blender.org/). 

1. Open it, import the garment from the `.obj` file. 
2. Then in `Layout` tab press `Tab` to go into the editing mode. 
3. Select all vertices you want to pin. 
4. Then, go into `Scripting` tab and execute the following piece of code there.

```python
import bpy
import bmesh

obj = bpy.context.active_object
bm = bmesh.from_edit_mesh(obj.data)    
obj = bpy.context.active_object; bm = bmesh.from_edit_mesh(obj.data)    ; selected = [i.index for i in bm.verts if i.select == True]; print(selected)

```


5. You will get a list of indices for the selected nodes.

Below are pinned indices for the garment `cindy_020_combined`, replace them with yours

In [5]:
pinned_indices = \
[   26,    27,    28,    36,    47,    56,    68,    69,    79,
          93,    94,   103,   121,   130,   151,   161,   185,   197,
         222,   237,   262,   279,   280,   307,   326,   353,   372,
         402,   422,   455,   456,   478,   513,   538,   576,   603,
         641,   670,   706,   738,   778,   812,   855,   891,   936,
         973,  1020,  1061,  1109,  1151,  1200,  1243,  1293,  1339,
        1390,  1439,  1492,  1543,  1597,  1651,  1706,  1761,  1821,
        1878,  1936,  1994,  2052,  2112,  2176,  2238,  2306,  2369,
        2439,  2508,  2576,  2644,  2713,  2719,  2784,  2858,  2926,
        3001,  3071,  3148,  3218,  3296,  3368,  3448,  3522,  3607,
        3687,  3770,  3850,  3934,  4016,  4102,  4187,  4269,  6567,
        6626,  7071,  7368,  7577,  7804,  7920,  8040,  8460,  8550,
        8632,  8992,  9065,  9137, 11694, 11695, 11700, 11706, 11712,
       11715, 11726, 11731, 11735, 11739, 11743, 11747, 11755, 11761,
       11770, 11779, 11792, 11797, 11802, 11814, 11819, 11824, 11829,
       11851, 11857]

add_pinned_verts(garment_dict_path, pinned_indices)

# Generate rollout for one sequence

Once we have created a garment dict file for our garment (or you can use one of the garments that are already under `DEFAULTS.data_root/aux_data/garment_dicts/`), we can generate a rollout sequence for them using a trained HOOD/ContourCraft model.

We provide 3 pretrained models and corresponding configuration files for each of them. The weights of the trained models are located in `DEFAULTS.data_root/trained_models`. The configuration files are in  `DEFAULTS.project_dir/configs`

| model file      | config name           | comments                                                                                                                                                                                                                            |
|-----------------|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| cvpr_submission | hood_cvpr                  | HOOD model used in the CVPR paper. No multi-layer simulation. Use it if you want to compare to the HOOD paper.                                                                                                                                                           |
| postcvpr        | hood_final              | Newer HOOD model trained using refactored code with minor bug fixes. No multi-layer simulation. Use it if you want to use HOOD model in a downstream task.|
| **contourcraft**        | **contourcraft**              | **Model from the ContourCraft paper. Can simulate multi-layer outfits**. |

## Choose pose sequence 

This repository supports inference over .npz sequences from CMU split of AMASS dataset.

You can download them [here](https://amass.is.tue.mpg.de/). Use gendered SMPL+H sequences if you want to use SMPL model and gendered SMPL-X sequences if you want to use SMPL-X one.


### create validation config and create `Runner` object

In [ ]:
from utils.validation import apply_material_params
from utils.validation import load_runner_from_checkpoint
from utils.arguments import load_params
from utils.common import move2device
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path


# Set material paramenters, see configs/cvpr.yaml for the training ranges for each parameter
material_dict = dict()
material_dict['density'] = 0.20022
material_dict['lame_mu'] = 23600.0
material_dict['lame_lambda'] = 44400
material_dict['bending_coeff'] = 3.962e-05


# ====================================================================================================

models_dir = Path(DEFAULTS.data_root) / 'trained_models'

# Choose the model and the configuration file

# config_name = 'hood_cvpr'
# checkpoint_path = models_dir / 'hood_cvpr.pth'

# config_name = 'hood_final'
# checkpoint_path = models_dir / 'hood_final.pth'

config_name = 'contourcraft'
checkpoint_path = models_dir / 'contourcraft.pth'


# ====================================================================================================


# load the config from .yaml file and load .py modules specified there
modules, experiment_config = load_params(config_name)

# modify the config to use it in validation 
experiment_config = apply_material_params(experiment_config, material_dict)

# load Runner object and the .py module it is declared in
runner_module, runner = load_runner_from_checkpoint(checkpoint_path, modules, experiment_config)


/local/home/agrigorev/Workdir/contourcraft_private/utils/validation.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(checkpoint_path)


### create one-sequence dataloader

Here you'll need to choose a garment by setting `garment_name` variable. The garment name should correspond to a `.pkl` file under `DEFAULTS.data_root/aux_data/garments_dicts/`

Note that it can also be a comma-separated list of garments, then they'll be combined into a single outfit. For example:
```
garment_name = 'smplx/cindy_020::bottom_skirt, smplx/cindy_020::top_blouse'
```

**To test for SMPL poses, use**

```
sequence_path =  'path/to/SMPLH/AMASS/CMU/01/01_01_poses.npz'
garment_name = 'longskirt'
sequence_loader = 'cmu_npz_smpl'
``` 

In [5]:
# file with the pose sequence
from utils.validation import create_postcvpr_one_sequence_dataloader

# If True, the SMPL(-X) poses are slightly modified to avoid hand-body self-penetrations. The technique is adopted from the code of SNUG 
separate_arms = True

# path to the pose sequence file
# CMU_path = 'path/to/AMASS/CMU'
CMU_path = '/data/agrigorev/00_Datasets/AMASS/smplx/CMU'
sequence_path =  Path(CMU_path) / '01/01_01_stageii.npz'

# name of the garment to simulate 
# should correspond to a pkl file under DEFAULTS.data_root/aux_data/garments_dicts/
garment_name = 'smplx/cindy_020_combined_test'

# It can be a comma-separated list of individual garments
# garment_name = 'smplx/cindy_020::bottom_skirt, smplx/cindy_020::top_blouse'


# gender of the body model, sould be the same as the one used to create the garment
gender = 'female'

# ====================================================================================================

# Choose the type of the pose sequence you want to use: 'cmu_npz_smpl', 'cmu_npz_smplx'

# to use AMASS SMPL-X pose sequence
sequence_loader = 'cmu_npz_smplx'

# to use AMASS SMPL pose sequence
# sequence_loader = 'cmu_npz_smpl



# ====================================================================================================


dataloader = create_postcvpr_one_sequence_dataloader(sequence_path, garment_name, sequence_loader=sequence_loader, 
                                            obstacle_dict_file=None, gender=gender)

In [6]:
sequence = next(iter(dataloader))
sequence = move2device(sequence, 'cuda:0')
trajectories_dict = runner.valid_rollout(sequence,  bare=True, n_steps=200)

100%|██████████| 200/200 [00:38<00:00,  5.22it/s]


In [ ]:
# Save the sequence to disk
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

Rollout saved into /data/agrigorev/02_Projects/ccraft_data/temp/output_hood_old.pkl


### write a video

Finally, we can render a video of the generated sequence with [aitviewer](https://github.com/eth-ait/aitviewer)

Or you can render it interactively using `python utils/show.py rollout_path=PATH_TO_SEQUENCE`

In [1]:
from utils.show import write_video 
from aitviewer.headless import HeadlessRenderer

# Careful!: creating more that one renderer in a single session causes an error
renderer = HeadlessRenderer()

/data/agrigorev/miniforge3/envs/ccraft/lib/python3.10/site-packages/trimesh/geometry.py:7: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  import scipy.sparse


In [2]:
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output.pkl'
out_video = Path(DEFAULTS.data_root) / 'temp' / 'output.mp4'
write_video(out_path, out_video, renderer)

NameError: name 'Path' is not defined